# Imports and Preprocessing

In [57]:
import os
import sys
import pandas as pd

folder_path = os.path.join(os.path.dirname(os.getcwd()), 'Data_Test_Multi_Raw')
print(folder_path)
file_names = ['data_test.csv', 'data_train.csv', 'target_test.csv', 'target_train.csv']

data_frames = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    data_frames.append(df)

data_test = data_frames[0]
data_train = data_frames[1]
target_test = data_frames[2]    
target_train = data_frames[3]

print(data_train.shape)
print(target_train.shape)

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\Data_Test_Multi_Raw
(800, 6)
(800, 6)


# Preprocessing

In [58]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Define preprocessing for numeric columns (scale them)
numeric_features = ['temperature']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Define preprocessing for categorical features (encode them)
categorical_features = ['location']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)],
    remainder='passthrough')

# Preprocessing on train data
X_train = preprocessor.fit_transform(data_train)

# Preprocessing on test data
X_test = preprocessor.transform(data_test)


print(X_train.shape)
print(X_test.shape)

for column in target_train.columns:
    print(target_train[column])

(800, 15)
(200, 15)
0      59
1      49
2      59
3      59
4      45
       ..
795    45
796    45
797    55
798    49
799    45
Name: product_1_demand, Length: 800, dtype: int64
0      73
1      60
2      73
3      73
4      55
       ..
795    55
796    55
797    69
798    61
799    55
Name: product_2_demand, Length: 800, dtype: int64
0      63
1      52
2      63
3      63
4      48
       ..
795    48
796    48
797    59
798    52
799    48
Name: product_3_demand, Length: 800, dtype: int64
0      35
1      29
2      35
3      35
4      28
       ..
795    28
796    28
797    31
798    28
799    28
Name: product_4_demand, Length: 800, dtype: int64
0      81
1      67
2      81
3      81
4      61
       ..
795    61
796    61
797    77
798    68
799    61
Name: product_5_demand, Length: 800, dtype: int64
0      84
1      69
2      84
3      84
4      63
       ..
795    63
796    63
797    79
798    70
799    63
Name: product_6_demand, Length: 800, dtype: int64


# Demand Forecasting with ANN Model

In [59]:
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.optimizers import Adam
from scipy.stats import reciprocal
import numpy as np

# Model creation function 
def create_model(n_hidden, n_neurons, learning_rate, activation):
    model = Sequential()
    # Input Layer
    model.add(Dense(n_neurons,input_dim=15, activation=activation))
    # Hidden Layer
    for _ in range(n_hidden):
        model.add(Dense(n_neurons, activation=activation))
    # Output Layer
    model.add(Dense(1))
    model.compile(loss="mean_squared_error", optimizer=Adam(learning_rate=learning_rate))
    return model

# Model builder function 
def model_builder(n_hidden, n_neurons, learning_rate, activation, batch_size, epochs):
    return KerasRegressor(build_fn=create_model, verbose=0, n_hidden=n_hidden, n_neurons=n_neurons, 
                          learning_rate=learning_rate, activation=activation, batch_size=batch_size, epochs=epochs)

# Define the hyperparameters to search for the Randomized Search
params = {
    "n_hidden": range(0, 15),
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(1e-4, 1e-2),
    "batch_size": [16, 32, 64, 128],
    "epochs": [10,15, 20, 25, 30],
    "activation": ['relu', 'sigmoid', 'tanh'] #
}



# Iterate over all items and predict values
target_pred_ANN = pd.DataFrame()
train_pred_ANN = pd.DataFrame()

for column in target_train.columns:
    # Create the model and define hyperparameter tuning with Randomized Search
    model_ANN = model_builder(1,30,3e-3,'relu', 32, 20)
    rnd_search_cv_ANN = RandomizedSearchCV(model_ANN, param_distributions=params, n_iter=10, cv=3, scoring='neg_mean_squared_error')

    # Fit the model on the training data
    rnd_search_cv_ANN.fit(X_train, target_train[column])
    print(rnd_search_cv_ANN.best_params_)

    # Predict the target values and add them as new columns to the DataFrames
    target_pred_ANN[column] = rnd_search_cv_ANN.predict(X_test)
    train_pred_ANN[column] = rnd_search_cv_ANN.predict(X_train)


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

{'activation': 'relu', 'batch_size': 32, 'epochs': 15, 'learning_rate': 0.000630794573045983, 'n_hidden': 0, 'n_neurons': 70}


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

{'activation': 'relu', 'batch_size': 64, 'epochs': 30, 'learning_rate': 0.00026789324798979467, 'n_hidden': 13, 'n_neurons': 23}


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

...

{'activation': 'relu', 'batch_size': 32, 'epochs': 25, 'learning_rate': 0.00011003077063814418, 'n_hidden': 10, 'n_neurons': 38}


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

{'activation': 'relu', 'batch_size': 64, 'epochs': 30, 'learning_rate': 0.0010176999510945874, 'n_hidden': 9, 'n_neurons': 37}


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

{'activation': 'relu', 'batch_size': 128, 'epochs': 25, 'learning_rate': 0.0036959593175434854, 'n_hidden': 13, 'n_neurons': 42}


# Demand Forecasting with DT (LightGBM)

In [60]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# Create the model and define the hyperparameters to search for the Randomized Search
model_DT = lgb.LGBMRegressor()
param_distribs = {
    "n_estimators": [50, 100, 200],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.1, 0.5],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
}

# Create the Randomized Search object
rnd_search_cv_DT = RandomizedSearchCV(model_DT, param_distribs, n_iter=10, cv=3, scoring='neg_mean_squared_error')

# Iterate over all items and predict values
target_pred_DT = pd.DataFrame()
train_pred_DT = pd.DataFrame()

for column in target_train:

    # Fit the model on the training data
    rnd_search_cv_DT.fit(X_train, target_train[column])
    print(rnd_search_cv_ANN.best_params_)

    # Get the best parameters and best estimator
    best_params = rnd_search_cv_DT.best_params_
    best_estimator = rnd_search_cv_DT.best_estimator_

    # Predict the target values and add them as new columns to the DataFrames
    target_pred_DT[column] = best_estimator.predict(X_test)
    train_pred_DT[column] = best_estimator.predict(X_train)



[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000400 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 80
[LightGBM] [Info] Number of data points in the train set: 533, number of used features: 14
[LightGBM] [Info] Start training from score 49.352720
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further split

# Solving Newsvendor (Non-parametric)

In [202]:
import numpy as np

# Initialize an empty list to store the final order quantities
final_order_quantities_ANN = pd.DataFrame(target_pred_ANN)
final_order_quantities_DT = pd.DataFrame(target_pred_DT)

# Parameters for multi-item newsvendor problem
prices_data = [0.3, 0.5, 0.6, 0.5, 0.5, 0.5] #price data
costs_data = [0.06, 0.06, 0.06, 0.06, 0.06, 0.06] #cost data
salvages_data = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01] #salvage data

alpha_data = [             #alpha data
    [0.0, 0.1, 0.05, 0.1, 0.05, 0.1],
    [0.15, 0.0, 0.1, 0.05, 0.05, 0.05],
    [0.1, 0.2, 0.0, 0.05, 0.1, 0.05],
    [0.05, 0.05, 0.05, 0.0, 0.15, 0.2],
    [0.1, 0.05, 0.15, 0.2, 0.0, 0.05],
    [0.05, 0.1, 0.05, 0.15, 0.1, 0.0]
]


In [203]:
import pulp as pl

def solve_milp(num_products, prices, costs, salvages, demands, substitution_rates):
    # Constants
    u = [a - b for a, b in zip(prices, costs)]
    o = [a - b for a, b in zip(costs, salvages)]
    M = 100  # Large number for big-M method

    # Create the problem variable to maximize the objective
    problem = pl.LpProblem("Single_Period_Newsvendor", pl.LpMaximize)

    # Decision variables
    q = pl.LpVariable.dicts("OrderQty", (range(num_products)), lowBound=0, cat='Continuous')
    y = pl.LpVariable.dicts("Overage", (range(num_products)), lowBound=0, cat='Continuous')
    v = pl.LpVariable.dicts("Underage", (range(num_products)), lowBound=0, cat='Continuous')
    z = pl.LpVariable.dicts("BinaryVar", (range(num_products)), cat='Binary')

    # Objective Function
    problem += pl.lpSum(u[i] * q[i] - (u[i] + o[i]) * y[i] for i in range(num_products))

    # Constraints
    for i in range(num_products):
        # Substitution demand constraint
        problem += y[i] >= q[i] - demands[i] - pl.lpSum(substitution_rates[j][i] * v[j] for j in range(num_products) if j != i)
        
        # Demand satisfaction constraints
        problem += v[i] <= demands[i] - q[i] + M * z[i]
        problem += v[i] >= demands[i] - q[i] - M * z[i]
        problem += v[i] <= demands[i] * (1 - z[i])

        # Non-negativity constraints
        problem += v[i] >= 0
        problem += y[i] >= 0


    # Solve the problem
    problem.solve()

    # Output the results
    order_quantities = [pl.value(q[i]) for i in range(num_products)]
    return order_quantities



In [204]:
# Solve problem with SAA (Sample Average Approximation) method - non-parametric

import pandas as pd
import numpy as np
from scipy.stats import norm

scenario_size = 100
demand_scenarios_ANN = pd.DataFrame()
demand_scenarios_DT = pd.DataFrame()

# Initialize an empty list to store the final order quantities
final_order_quantities_ANN_SAA = []
final_order_quantities_DT_SAA = []

# Calculate the forecast error for ANN on the prediction of the training data
forecast_error_ANN = target_train - train_pred_ANN
forecast_error_DT = target_train - train_pred_DT

# Loop over each week in data_test
for i in range(10): #len(X_test)
  
    # Initialize a list to store the solutions for each scenario
    saa_solutions_ANN = []
    saa_solutions_DT = []

    # For each demand scenario, solve the newsvendor problem
    for k in range(scenario_size):
        # Create the demand scenario by adding a random forecast error to the prediction
        random_row_index = np.random.randint(0, len(forecast_error_ANN))
        demand_scenarios_ANN = target_pred_ANN.iloc[i] + forecast_error_ANN.iloc[random_row_index]
        demand_scenarios_DT = target_pred_DT.iloc[i] + forecast_error_DT.iloc[random_row_index]
        demand_scenarios_ANN = demand_scenarios_ANN.to_list()
        demand_scenarios_DT = demand_scenarios_DT.to_list()

        # Calculate the solution for this scenario
        solution_ANN = solve_milp(6, prices_data, costs_data, salvages_data, demand_scenarios_ANN, alpha_data)
        solution_DT = solve_milp(6, prices_data, costs_data, salvages_data, demand_scenarios_DT, alpha_data)
      
        # Store the solution for this scenario
        saa_solutions_ANN.append(solution_ANN)
        saa_solutions_DT.append(solution_DT)

    # Average the solutions to get the final allocation
    final_allocation_ANN = pd.DataFrame(saa_solutions_ANN).mean().tolist()
    final_allocation_DT = pd.DataFrame(saa_solutions_DT).mean().tolist()

    # Store the final order quantities
    final_order_quantities_ANN_SAA.append(final_allocation_ANN)
    final_order_quantities_DT_SAA.append(final_allocation_DT)


In [205]:
# Solve problem model-based

import pandas as pd
import numpy as np
from scipy.stats import norm

scenario_size = 100
demand_scenarios_ANN = pd.DataFrame()
demand_scenarios_DT = pd.DataFrame()

# Initialize an empty list to store the final order quantities
final_order_quantities_ANN_model = []
final_order_quantities_DT_model = []

# Calculate the forecast error for ANN on the prediction of the training data
forecast_error_ANN = target_train - train_pred_ANN
forecast_error_DT = target_train - train_pred_DT
forecast_error_ANN_std = forecast_error_ANN.std().tolist()
forecast_error_DT_std = forecast_error_DT.std().tolist()

# Loop over each week in data_test
for i in range(10): #len(X_test)
  
    # Initialize a list to store the solutions for each scenario
    saa_solutions_ANN = []
    saa_solutions_DT = []

    # For each demand scenario, solve the newsvendor problem
    for k in range(scenario_size):
        # Create the demand scenario by adding a random forecast error based on the standard deviation of the forecast error
        demand_scenarios_ANN = target_pred_ANN.iloc[i] + np.random.normal(0, forecast_error_ANN_std)
        demand_scenarios_DT = target_pred_DT.iloc[i] + np.random.normal(0, forecast_error_DT_std)
        demand_scenarios_ANN = demand_scenarios_ANN.to_list()
        demand_scenarios_DT = demand_scenarios_DT.to_list()

        # Calculate the solution for this scenario
        solution_ANN = solve_milp(6, prices_data, costs_data, salvages_data, demand_scenarios_ANN, alpha_data)
        solution_DT = solve_milp(6, prices_data, costs_data, salvages_data, demand_scenarios_DT, alpha_data)
      
        # Store the solution for this scenario
        saa_solutions_ANN.append(solution_ANN)
        saa_solutions_DT.append(solution_DT)

    # Average the solutions to get the final allocation
    final_allocation_ANN = pd.DataFrame(saa_solutions_ANN).mean().tolist()
    final_allocation_DT = pd.DataFrame(saa_solutions_DT).mean().tolist()

    # Store the final order quantities
    final_order_quantities_ANN_model.append(final_allocation_ANN)
    final_order_quantities_DT_model.append(final_allocation_DT)


# Evaluation

In [221]:
# Loop over each week in target_test
overall_costs_ANN = 0
overall_costs_DT = 0
overall_costs_ANN_model = 0
overall_costs_DT_model = 0

for j in range(6):
    for i in range(10):

        # Calculate understock and overstock costs
        cost_ANN = 0
        cost_DT = 0
        cost_ANN_model = 0
        cost_DT_model = 0

        
        # Costs for non-parametric
        if final_order_quantities_ANN_SAA[i][j] < target_test.values[i][j]:
            cost_ANN = (prices_data[j] - costs_data[j]) * (target_test.values[i][j] - np.round(final_order_quantities_ANN_SAA[i][j]))
        if final_order_quantities_ANN_SAA[i][j] > target_test.values[i][j]:
            cost_ANN = (costs_data[j] - salvages_data[j]) * (np.round(final_order_quantities_ANN_SAA[i][j]) - target_test.values[i][j])
        if final_order_quantities_DT_SAA[i][j] < target_test.values[i][j]:
            cost_DT = (prices_data[j] - costs_data[j]) * (target_test.values[i][j] - np.round(final_order_quantities_DT_SAA[i][j]))
        if final_order_quantities_DT_SAA[i][j] > target_test.values[i][j]:
            cost_DT = (costs_data[j] - salvages_data[j]) * (np.round(final_order_quantities_DT_SAA[i][j]) - target_test.values[i][j])
        # Costs for model-based
        if final_order_quantities_ANN_model[i][j] < target_test.values[i][j]:
            cost_ANN_model = (prices_data[j] - costs_data[j]) * (target_test.values[i][j] - np.round(final_order_quantities_ANN_model[i][j]))
        if final_order_quantities_ANN_model[i][j] > target_test.values[i][j]:
            cost_ANN_model = (costs_data[j] - salvages_data[j]) * (np.round(final_order_quantities_ANN_model[i][j]) - target_test.values[i][j])
        if final_order_quantities_DT_model[i][j] < target_test.values[i][j]:
            cost_DT_model = (prices_data[j] - costs_data[j]) * (target_test.values[i][j] - np.round(final_order_quantities_DT_model[i][j]))
        if final_order_quantities_DT_model[i][j] > target_test.values[i][j]:
            cost_DT_model = (costs_data[j] - salvages_data[j]) * (np.round(final_order_quantities_DT_model[i][j]) - target_test.values[i][j])

        # Calculate the total costs for the week
        overall_costs_ANN += cost_ANN
        overall_costs_DT += cost_DT
        overall_costs_ANN_model += cost_ANN_model
        overall_costs_DT_model += cost_DT_model
   

# Print the overall profit
print('Overall costs for ANN (non-parametric): ', int(overall_costs_ANN))
print('Overall costs for DT (non-parametric): ', int(overall_costs_DT))
print('Overall costs for ANN (model-based): ', int(overall_costs_ANN_model))
print('Overall costs for DT (model-based): ', int(overall_costs_DT_model))

i: 0 j: 0
i: 1 j: 0
i: 0 j: 1
i: 1 j: 1
i: 0 j: 2
i: 1 j: 2
i: 0 j: 3
i: 1 j: 3
i: 0 j: 4
i: 1 j: 4
i: 0 j: 5
i: 1 j: 5
Overall costs for ANN (non-parametric):  51
Overall costs for DT (non-parametric):  3
Overall costs for ANN (model-based):  49
Overall costs for DT (model-based):  3
